In [3]:
data = '''#.#####################
#.......#########...###
#######.#########.#.###
###.....#.>.>.###.#.###
###v#####.#v#.###.#.###
###.>...#.#.#.....#...#
###v###.#.#.#########.#
###...#.#.#.......#...#
#####.#.#.#######.#.###
#.....#.#.#.......#...#
#.#####.#.#.#########v#
#.#...#...#...###...>.#
#.#.#v#######v###.###v#
#...#.>.#...>.>.#.###.#
#####v#.#.###v#.#.###.#
#.....#...#...#.#.#...#
#.#########.###.#.#.###
#...###...#...#...#.###
###.###.#.###v#####v###
#...#...#.#.>.>.#.>.###
#.###.###.#.###.#.#v###
#.....###...###...#...#
#####################.#'''

In [4]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import networkx as nx

In [76]:
from aocd import get_data
data = get_data(day=23, year=2023)

In [5]:
grid = {(i, j): n for j, line in enumerate(data.split('\n')) for i, n in enumerate(line)}
G = nx.DiGraph()
for i, j in grid.keys():
    if grid[(i,j)] == '#':
        continue # not a valid move
    if grid[(i,j)] == 'v':
        G.add_edge((i,j), (i,j+1))
    elif grid[(i,j)] == '>':
        G.add_edge((i,j), (i+1,j))
    else:
        for dx, dy, dd in {(1,0,1),(0,1,-1),(-1,0,1),(0,-1,-1)} - {(i,j),(-i,-j)}:
            a, b = i+dx, j+dy
            if (a,b) in grid:
                c = grid[(a,b)]
                if c == '#':
                    continue
                G.add_edge((i,j), (a,b))

In [6]:
start = (1,0)
end = (data.count('\n')-1, data.index('\n')-1)
max([ len(p)-1 for p in nx.all_simple_paths(G, start, end) ])

94

In [29]:
for e in nx.strongly_connected_components(G):
    SG = G.subgraph(e)
    print(SG)
    ends = []
    for o in SG:
        if nx.degree(SG, o) == 2:
            ends.append(o)
        #print(o, nx.degree(SG, o), SG.edges(o))
    print(ends, )
    #print(SG.degree([(19,20)]))
    break

DiGraph with 5 nodes and 8 edges
[(19, 20), (21, 22)]


In [56]:
#list(nx.strongly_connected_components(G))
NG = nx.condensation(G)
NG.edges

OutEdgeView([(1, 0), (2, 1), (3, 2), (4, 3), (5, 3), (6, 1), (7, 6), (8, 7), (9, 5), (9, 8), (10, 9), (11, 4), (11, 10), (12, 11), (13, 9), (14, 7), (15, 13), (15, 14), (16, 15), (17, 12), (17, 16), (18, 17)])

In [30]:
NG = nx.DiGraph()
for u,v,d in nx.dfs_labeled_edges(G, start):
    match d:
        case 'forward': # u has been visited but v has not
            pass
        case 'nontree': # both u and v have been visited but the edge is not in the DFS tree
            pass
        case 'reverse': # both u and v have been visited and the edge is in the DFS tree
            pass
    #print(u,v,d)

In [95]:
G = nx.DiGraph()
visited = {}

queue = [tuple(start)]
while queue:
    current = queue.pop(0)
    visited[current] = 1
    i, j = current
    for dx, dy, dd in {(1,0,1),(0,1,-1),(-1,0,1),(0,-1,-1)} - {(i,j),(-i,-j)}:
        a, b = i+dx, j+dy
        if (a,b) in grid and (a,b) not in visited:
            visited[(a,b)] = 1
            c = grid[(a,b)]
            if c == '#':
                continue
            #print('add', (i,j), (a,b))
            G.add_edge((i,j), (a,b))
            queue.append((a,b))

[ len(x) for x in nx.all_simple_paths(G, start, end) ]

[75]

In [49]:
NG = nx.DiGraph()

visited = []
start = tuple(start)
current = tuple(start)
while nx.degree(G, current) == 2:
    visited.append(current)
    current = list(G.successors(current))[0]
    print('to', current, nx.degree(G, current))


to (1, 1) 4


In [50]:
G.edges((1,1))

OutEdgeDataView([((1, 1), (1, 0)), ((1, 1), (2, 1))])

In [44]:
list(G.successors(start))

[(1, 1)]

In [27]:
NG = nx.DiGraph()
weights = {}
def construct(G, NG, current, path=None, visited=[]):
    visited.append(current)
    path = path if path else []
    path.append(tuple(current))
    n = [ e for e in G.edges(current) if e[1] not in visited ]
    if len(n) == 1:
        construct(G, NG, n[0][1], path, visited=visited)
    elif len(n) > 1:
        #if path[0] == (4,5):
        #    print(current, path)
        NG.add_edge(path[0], tuple(current), weight=len(path)+1)
        for e in n:
            NG.add_edge(current, e[1], weight=1)
            construct(G, NG, e[1], visited=visited)
    else:
        NG.add_edge(path[0], tuple(current), weight=len(path)+1)
construct(G, NG, start)

In [31]:
NG.edges

OutEdgeView([((1, 0), (3, 5)), ((3, 5), (4, 5)), ((3, 5), (3, 6)), ((4, 5), (11, 3)), ((11, 3), (12, 3)), ((11, 3), (11, 4)), ((12, 3), (21, 11)), ((21, 11), (21, 12)), ((21, 11), (20, 11)), ((21, 12), (19, 19)), ((19, 19), (19, 20)), ((19, 19), (18, 19)), ((19, 20), (21, 22)), ((18, 19), (18, 19)), ((20, 11), (20, 11)), ((11, 4), (13, 13)), ((13, 13), (14, 13)), ((13, 13), (13, 14)), ((13, 13), (12, 13)), ((14, 13), (19, 11)), ((13, 14), (13, 19)), ((13, 19), (14, 19)), ((13, 19), (12, 19)), ((14, 19), (17, 19)), ((12, 19), (12, 19)), ((12, 13), (12, 13)), ((3, 6), (5, 13)), ((5, 13), (6, 13)), ((5, 13), (5, 14)), ((6, 13), (11, 13)), ((5, 14), (11, 19))])

In [32]:
for p in nx.all_simple_paths(NG, start, end):
    print(p)
    ptotal = 0
    for l, r in zip(p, p[1:]):
        print(' ', l, r, NG.get_edge_data(l, r)['weight'])
        ptotal += NG.get_edge_data(l, r)['weight']
    print(ptotal)


[(1, 0), (3, 5), (4, 5), (11, 3), (12, 3), (21, 11), (21, 12), (19, 19), (19, 20), (21, 22)]
  (1, 0) (3, 5) 17
  (3, 5) (4, 5) 1
  (4, 5) (11, 3) 23
  (11, 3) (12, 3) 1
  (12, 3) (21, 11) 31
  (21, 11) (21, 12) 1
  (21, 12) (19, 19) 11
  (19, 19) (19, 20) 1
  (19, 20) (21, 22) 6
92


In [29]:
NG.get_edge_data((1,0), (3, 5))['weight']

17

In [30]:
grid[(21, 22)]

'.'